In [1]:
import numpy as np
import pandas as pd
import glob
import torch
import matplotlib.pyplot as plt
import random
import pickle
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
from   sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from   sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding, MDS, SpectralEmbedding
from   sklearn.preprocessing import StandardScaler
#!{sys.executable} -m pip install scikit-learn-extra






In [3]:
reference_path = '/mnt/data/CAMI/DNABERT/output/'

tensors = []
contig_names = []
for filename in glob.glob('/mnt/data/CAMI/DNABERT/output/*.pickle'):
    with open(filename, 'rb') as f:
        x = pickle.load(f)
        tensors.extend(list(x.values()))
        contig_names.extend(list(x.keys()))
concat_tensors = torch.cat(tensors,0)

In [4]:
taxonomy = '/mnt/data/CAMI/data/short_read_oral/taxonomy.tsv'
contig_to_genome = '/mnt/data/CAMI/data/short_read_oral/reformatted_manually_combined_gsa_mapping.tsv'

contig_to_genome_df = pd.read_csv(contig_to_genome, sep='\t', header=None)
contig_to_genome_df = contig_to_genome_df.rename(columns={0: 'contig_name', 1: 'genome'})

taxonomy_df = pd.read_csv(taxonomy, sep='\t', header = None)
taxonomy_df = taxonomy_df.rename(columns={0: 'genome', 1: 'species', 2: 'genus'})

In [5]:
merged_df = pd.merge(contig_to_genome_df, taxonomy_df, how="left", on=["genome"])
merged_df

,contig_name,genome,2,3,4,species,genus
0,S13C144379,OTU_97.10732.1,CP014228.1,848814,849755,Actinomyces radicidentis,Actinomyces
1,S13C4155,OTU_97.27895.0,CP010906.1,758589,760618,Campylobacter jejuni,Campylobacter
2,S13C140957,OTU_97.37776.1,CP002094.1,2081039,2081268,Lactococcus lactis,Lactococcus
3,S13C24289,OTU_97.34522.1,AM295007.1,1679584,1680153,Streptococcus pyogenes,Streptococcus
4,S13C116303,OTU_97.23.0,LN831051.1,2043591,2043855,Streptococcus pneumoniae,Streptococcus
...,...,...,...,...,...,...,...
2964252,S8C75706,OTU_97.28883.0,CP000921.1,1377239,1379025,Streptococcus pneumoniae,Streptococcus
2964253,S8C46428,OTU_97.39134.0,FM204883.1,609148,609430,Streptococcus equi,Streptococcus
2964254,S8C149508,OTU_97.39522.0,CP012075.1,150371,152780,Prevotella fusca,Prevotella
2964255,S8C112712,OTU_97.28231.0,HE858529.1,2154398,2155309,Streptococcus dysgalactiae,Streptococcus


In [6]:
contig_lengths = "/mnt/data/CAMI/DNABERT/contig_lengths.pickle"
with open(contig_lengths,'rb') as f:
    contig_lengths = pickle.load(f)
    
contig_lengths = {k[1:]: v for k, v in contig_lengths.items()}
merged_df["contig_lengths"] = merged_df["contig_name"].map(contig_lengths)
merged_df

,contig_name,genome,2,3,4,species,genus,contig_lengths
0,S13C144379,OTU_97.10732.1,CP014228.1,848814,849755,Actinomyces radicidentis,Actinomyces,NaN
1,S13C4155,OTU_97.27895.0,CP010906.1,758589,760618,Campylobacter jejuni,Campylobacter,NaN
2,S13C140957,OTU_97.37776.1,CP002094.1,2081039,2081268,Lactococcus lactis,Lactococcus,NaN
3,S13C24289,OTU_97.34522.1,AM295007.1,1679584,1680153,Streptococcus pyogenes,Streptococcus,NaN
4,S13C116303,OTU_97.23.0,LN831051.1,2043591,2043855,Streptococcus pneumoniae,Streptococcus,NaN
...,...,...,...,...,...,...,...,...
2964252,S8C75706,OTU_97.28883.0,CP000921.1,1377239,1379025,Streptococcus pneumoniae,Streptococcus,NaN
2964253,S8C46428,OTU_97.39134.0,FM204883.1,609148,609430,Streptococcus equi,Streptococcus,NaN
2964254,S8C149508,OTU_97.39522.0,CP012075.1,150371,152780,Prevotella fusca,Prevotella,NaN
2964255,S8C112712,OTU_97.28231.0,HE858529.1,2154398,2155309,Streptococcus dysgalactiae,Streptococcus,NaN


In [7]:
contig_name_to_idx = {v[1:]: i for i, v in enumerate(contig_names)}
merged_df["contig_idx"] = merged_df["contig_name"].apply(lambda x: contig_name_to_idx.get(x))
non_aligned = ~merged_df["contig_idx"].isnull()
aligned_tensor_df = merged_df[non_aligned]
aligned_tensor_df

,contig_name,genome,2,3,4,species,genus,contig_lengths,contig_idx
1667473,S19C259815,OTU_97.41539.0,CP000260.1,1522037,1522883,Streptococcus pyogenes,Streptococcus,847.0,450415.0
1667474,S19C345587,OTU_97.11620.0,CP016442.1,3161170,3161457,Burkholderia stabilis,Burkholderia,288.0,291286.0
1667475,S19C625314,OTU_97.39134.1,CP003858.1,1362831,1363769,Streptococcus intermedius,Streptococcus,939.0,390114.0
1667476,S19C433562,OTU_97.14178.0,CP003368.1,1390173,1390609,Prevotella dentalis,Prevotella,437.0,656162.0
1667477,S19C496262,OTU_97.28043.0,CP016912.1,844138,844733,Burkholderia pseudomallei,Burkholderia,596.0,75861.0
...,...,...,...,...,...,...,...,...,...
2347903,S19C71531,OTU_97.40690.0,CP009147.1,957418,979750,Burkholderia mallei,Burkholderia,22333.0,603531.0
2347904,S19C197686,OTU_97.41539.0,CP000260.1,796679,797632,Streptococcus pyogenes,Streptococcus,954.0,240885.0
2347905,S19C294810,OTU_97.29395.1,CP000381.1,1154419,1154669,Neisseria meningitidis,Neisseria,251.0,36709.0
2347906,S19C452300,OTU_97.8703.1,CP017050.1,2539173,2539461,Burkholderia pseudomallei,Burkholderia,289.0,4199.0


In [8]:
column = aligned_tensor_df['contig_lengths']
max_value = column.max()
print(max_value)

3018058.0


In [9]:
aligned_tensor_index_df = aligned_tensor_df.astype({"contig_idx": int})
aligned_tensor_index_df = aligned_tensor_index_df.set_index("contig_idx").sort_index()
aligned_tensor_index_df

,contig_name,genome,2,3,4,species,genus,contig_lengths
contig_idx,,,,,,,,
0,S19C531701,OTU_97.28043.1,CP012517.1,1058437,1058708,Burkholderia pseudomallei,Burkholderia,272.0
1,S19C531702,OTU_97.27554.0,FN665654.1,303528,304852,Clostridioides difficile,Clostridioides,1325.0
2,S19C531703,OTU_97.9324.1,CP008782.1,1755368,1755617,Burkholderia pseudomallei,Burkholderia,250.0
3,S19C531704,OTU_97.27207.0,AP013072.1,1019916,1020409,Streptococcus anginosus,Streptococcus,494.0
4,S19C531705,OTU_97.934.0,CP008695.1,345079,345321,Streptococcus pyogenes,Streptococcus,243.0
...,...,...,...,...,...,...,...,...
680396,S19C275396,OTU_97.43518.0,CP019469.1,2197272,2197996,Clostridioides difficile,Clostridioides,725.0
680397,S19C275397,OTU_97.13253.0,LT671674.1,1375069,1375372,Streptococcus suis,Streptococcus,304.0
680398,S19C275398,OTU_97.8361.0,CP003087.1,2947729,2948014,Burkholderia sp. YI23,Burkholderia,286.0


In [10]:
# Filter all contigs > 512

print(len(aligned_tensor_index_df))
filtered_512_df = aligned_tensor_index_df[aligned_tensor_index_df["contig_lengths"] < 512]
print(len(filtered_512_df))

680401
431137


In [ ]:
def plot_kmeans():
    all_concat_np = concat_tensors.detach().numpy()
    
    pca = PCA(n_components=2)
    pca.fit(all_concat_np)
    projection = pca.transform(all_concat_np)
    
    
    from sklearn.cluster import KMeans
    wcss = []
    for i in range(100,700):
        kmeans_pca = KMeans(n_clusters=i, init='k-means++', random_state=None)
        kmeans_pca.fit(projection)
        wcss.append(kmeans_pca.inertia_)

    plt.plot(range(100,700), wcss, marker = 'o', linestyle = '--')
    
    #kmeans_pca = KMeans(n_clusters = 3, init = 'k-means++', random_state=None).fit(projection)
    #print(kmeans_pca.cluster_centers_)
    #print(kmeans_pca.inertia_)
    
    #from sklearn_extra.cluster import KMedoids
    #ss = []
    #for i in range(1,10):
        #kmedoids_pca = KMedoids(n_clusters=2, random_state=0)
        #kmedoids_pca.fit(projection)
        #ss.append(kmedoids_pca.inertia_)
    
    #plt.plot(range(1,10), ss, marker = 'o', linestyle = '--')

    #kmedoids_pca = KMedoids(n_clusters=2, random_state=0).fit(projection)
    #print(kmedoids_pca.cluster_centers_)
    #print(kmedoids_pca.inertia_)
    
plot_kmeans()

In [ ]:
MIN_SIZE_OF_GROUP = 100
NUM_TO_GROUPS_TO_PLOT = 10
GROUP_KEY = "species"

genome_group_indices = []
i = 0
groups = list(filtered_512_df.groupby(GROUP_KEY))
random.shuffle(groups)
for x_name, x in groups:
    if i >= 10:
        break
        
    group_size = len(x)
    if group_size < MIN_SIZE_OF_GROUP or group_size > 1000:
        continue
        
    genome_group_indices.extend(x.index.tolist())
    i += 1

print(len(genome_group_indices))

In [ ]:
def plot_pca():
    all_concat_np = concat_tensors.detach().numpy()
    genome_group_np = all_concat_np[genome_group_indices]
    
    pca = PCA(n_components=2)
    pca.fit(genome_group_np)
    projection = pca.transform(genome_group_np)
    
    genome_to_color_id = {k: i for k, i in zip(filtered_512_df.loc[genome_group_indices][GROUP_KEY].unique(), range(10))}
    targets = filtered_512_df.loc[genome_group_indices][GROUP_KEY].apply(lambda x: genome_to_color_id[x]).tolist()
    labels = filtered_512_df.loc[genome_group_indices][GROUP_KEY].unique().tolist()
    plt.figure(figsize=(10, 10))
    scatter = plt.scatter(projection[:, 0], projection[:, 1], alpha=0.9, s=5.0, c=targets, cmap='tab10')
    plt.legend(handles=scatter.legend_elements()[0], labels=labels)
    
plot_pca()

In [ ]:
def plot_kmeans():
    all_concat_np = concat_tensors.detach().numpy()
    genome_group_np = all_concat_np[genome_group_indices]
    
    pca = PCA(n_components=2)
    pca.fit(genome_group_np)
    projection = pca.transform(genome_group_np)
    
    
    from sklearn.cluster import KMeans
    wcss = []
    for i in range(1,10):
        kmeans_pca = KMeans(n_clusters=i, init='k-means++', random_state=None)
        kmeans_pca.fit(projection)
        wcss.append(kmeans_pca.inertia_)

    plt.plot(range(1,10), wcss, marker = 'o', linestyle = '--')
    
    kmeans_pca = KMeans(n_clusters = 3, init = 'k-means++', random_state=None).fit(projection)
    print(kmeans_pca.cluster_centers_)
    print(kmeans_pca.inertia_)
    
    from sklearn_extra.cluster import KMedoids
    ss = []
    for i in range(1,10):
        kmedoids_pca = KMedoids(n_clusters=2, random_state=0)
        kmedoids_pca.fit(projection)
        ss.append(kmedoids_pca.inertia_)
    
    plt.plot(range(1,10), ss, marker = 'o', linestyle = '--')

    kmedoids_pca = KMedoids(n_clusters=2, random_state=0).fit(projection)
    print(kmedoids_pca.cluster_centers_)
    print(kmedoids_pca.inertia_)
    
plot_kmeans()



In [ ]:
def plot_tsne():    
    all_concat_np = concat_tensors.detach().numpy()
    genome_group_np = all_concat_np[genome_group_indices]
    
    #NUM_SUBSAMPLE = 10000
    subsample_indices = list(range(len(genome_group_np)))
    #random.shuffle(subsample_indices)
    # subsample_indices =  subsample_indices[:NUM_SUBSAMPLE]
    
    subsampled_genome_groups_np = genome_group_np[subsample_indices]
    tsne = TSNE(n_components=2, perplexity=30)
    projection = tsne.fit_transform(subsampled_genome_groups_np)
    
    genome_to_color_id = {k: i for k, i in zip(filtered_512_df.loc[genome_group_indices].iloc[subsample_indices][GROUP_KEY].unique(), range(10))}
    targets = filtered_512_df.loc[genome_group_indices].iloc[subsample_indices][GROUP_KEY].apply(lambda x: genome_to_color_id[x]).tolist()
    labels = filtered_512_df.loc[genome_group_indices][GROUP_KEY].unique().tolist()
    plt.figure(figsize=(10, 10))
    scatter = plt.scatter(projection[:, 0], projection[:, 1], alpha=0.9, s=3.0, c=targets, cmap='tab10')
    plt.legend(handles=scatter.legend_elements()[0], labels=labels)

plot_tsne()

In [ ]:
import sys
import vamb

def create_contig_file_list(path_to_contig_file):
    contig_list = []
    with open(path_to_contig_file, 'r') as fp:
        lines = fp.readlines()
        for line in lines:
            line = line.rstrip()
            contig_list.append(line)
    return contig_list

mincontiglength=10
contig_file_list='/mnt/data/CAMI/vamb/workflow/contigs.txt'
file_list = create_contig_file_list(contig_file_list)

tnfs_per_fasta = []
contignames_per_fasta = []
for fasta in file_list:
    with vamb.vambtools.Reader(fasta, 'rb') as tnffile:
        tnfs, contignames, contiglengths = vamb.parsecontigs.read_contigs(tnffile, minlength=mincontiglength)
        tnfs_per_fasta.append(tnfs)
        contignames_per_fasta.extend(contignames)

tnfs_per_fasta = np.concatenate(tnfs_per_fasta)

In [ ]:
print(len(contignames_per_fasta))
print(len(tnfs))

In [ ]:
# contignames to be sorted like filtered_512_df

contig_name_to_idx = {v: i for i, v in enumerate(contignames_per_fasta)}
vamb_aligned_tensor_df["contig_vamb_idx"] = aligned_tensor_df["contig_name"].map(contig_name_to_idx)
vamb_aligned_tensor_df

In [ ]:
non_aligned = ~vamb_aligned_tensor_df["contig_vamb_idx"].isnull()
vam_aligned_reset_tensor_df = vamb_aligned_tensor_df[non_aligned]
vamb_aligned_tensor_df = vamb_aligned_tensor_df.astype({"contig_vamb_idx": int})
vamb_aligned_tensor_df = vamb_aligned_tensor_df.astype({"contig_idx": int})
vamb_aligned_tensor_df

In [ ]:
vamb_aligned_tensor_df_2 = vamb_aligned_tensor_df.set_index("contig_idx").sort_index()
vamb_aligned_tensor_df_2

In [ ]:
print(len(vamb_aligned_tensor_df))
vamb_filtered_512_df = vamb_aligned_tensor_df_2[vamb_aligned_tensor_df_2["contig_lengths"] < 512]

print(len(vamb_filtered_512_df))
vamb_filtered_512_df

In [ ]:
MIN_SIZE_OF_GROUP = 100
MAX_SIZE_OF_GROUP = 1000
NUM_TO_GROUPS_TO_PLOT = 10
GROUP_KEY = "species"

genome_group_indices = []
i = 0
groups = list(vamb_filtered_512_df.groupby(GROUP_KEY))
random.shuffle(groups)
for x_name, x in groups:
    if i >= 10:
        break
        
    group_size = len(x)
    if group_size < MIN_SIZE_OF_GROUP or group_size > MAX_SIZE_OF_GROUP:
        continue
        
    genome_group_indices.extend(x.index.tolist())
    i += 1

print(len(genome_group_indices))

In [ ]:
def plot_pca_matrix(matrix):
    pca = PCA(n_components=2)
    pca.fit(matrix)
    projection = pca.transform(matrix)
    
    genome_to_color_id = {k: i for k, i in zip(vamb_filtered_512_df.loc[genome_group_indices][GROUP_KEY].unique(), range(10))}
    targets = vamb_filtered_512_df.loc[genome_group_indices][GROUP_KEY].apply(lambda x: genome_to_color_id[x]).tolist()
    labels = vamb_filtered_512_df.loc[genome_group_indices][GROUP_KEY].unique().tolist()
    plt.figure(figsize=(10, 10))
    scatter = plt.scatter(projection[:, 0], projection[:, 1], alpha=0.9, s=5.0, c=targets, cmap='tab10')
    plt.legend(handles=scatter.legend_elements()[0], labels=labels)

tnf_matrix = tnfs_per_fasta[vamb_filtered_512_df.loc[genome_group_indices]["contig_vamb_idx"].tolist()]
plot_pca_matrix(tnf_matrix)

all_concat_np = concat_tensors.detach().numpy()
genome_group_np = all_concat_np[genome_group_indices]
plot_pca_matrix(genome_group_np)

tnf_matrix = (tnf_matrix) / (tnf_matrix.std())
genome_group_np = (genome_group_np) / (genome_group_np.std())

cat_tnf_dna = np.concatenate([genome_group_np, tnf_matrix], axis=-1)
plot_pca_matrix(cat_tnf_dna)

In [ ]:
def plot_tsne_matrix(matrix):    
    tsne = TSNE(n_components=2, perplexity=30)
    projection = tsne.fit_transform(matrix)
    
    genome_to_color_id = {k: i for k, i in zip(vamb_filtered_512_df.loc[genome_group_indices][GROUP_KEY].unique(), range(10))}
    targets = vamb_filtered_512_df.loc[genome_group_indices][GROUP_KEY].apply(lambda x: genome_to_color_id[x]).tolist()
    labels = vamb_filtered_512_df.loc[genome_group_indices][GROUP_KEY].unique().tolist()
    plt.figure(figsize=(10, 10))
    scatter = plt.scatter(projection[:, 0], projection[:, 1], alpha=0.9, s=3.0, c=targets, cmap='tab10')
    plt.legend(handles=scatter.legend_elements()[0], labels=labels)

tnf_matrix = tnfs_per_fasta[vamb_filtered_512_df.loc[genome_group_indices]["contig_vamb_idx"].tolist()]
plot_tsne_matrix(tnf_matrix)

all_concat_np = concat_tensors.detach().numpy()
genome_group_np = all_concat_np[genome_group_indices]
plot_tsne_matrix(genome_group_np)

tnf_matrix = (tnf_matrix) / (tnf_matrix.std())
genome_group_np = (genome_group_np) / (genome_group_np.std())
cat_tnf_dna = np.concatenate([genome_group_np, tnf_matrix], axis=-1)
plot_tsne_matrix(cat_tnf_dna)

In [ ]:
#torch.load('/mnt/data/CAMI/DNABERT/pretrained_models/4-new-12w-0/pytorch_model.bin', map_location=torch.device('cpu'))

In [ ]:
import torch 
from transformers import BertTokenizer, BertforSequenceClassification